In [6]:
import pandas as pd
import pickle
import numpy as np
import datetime 
from os.path import join as pjoin
import os
#import argparse
#import yaml

In [1]:
# # if package is not properly loaded, check env path 
# import sys

# print(sys.path)

# from openpyxl import Workbook


['/home/dalab5/Projects/MA_refined', '/home/dalab5/miniconda3/envs/GNN/lib/python38.zip', '/home/dalab5/miniconda3/envs/GNN/lib/python3.8', '/home/dalab5/miniconda3/envs/GNN/lib/python3.8/lib-dynload', '', '/home/dalab5/miniconda3/envs/GNN/lib/python3.8/site-packages', '/home/dalab5/miniconda3/envs/GNN/lib/python3.8/site-packages/IPython/extensions', '/home/dalab5/.ipython']


In [10]:
# parser = argparse.ArgumentParser(description='MA basic setting')
# parser.add_argument('--config', default='./config.yaml')

# #global args
# args = parser.parse_args()
# with open(args.config) as f:
#     config = yaml.load(f)
    
# for key in config:
#     for k, v in config[key].items():
#         setattr(args, k, v)

In [8]:
tmp_data_path =  './data/tmp'
s_year = 1997
e_year = 2020

# import data

3 groups of data
- bridge 1: wrds bridge
- bridge 2: evans bridge
- SDC MA data

In [127]:
# bridge 1
wrds_bridge = pd.read_csv('./data/wrds_bridge.csv', header=0, engine=None)

In [128]:
# rename 

wrds_bridge.rename(columns={x: x.replace(' ', '_').upper().strip() for x in wrds_bridge.columns}, inplace=True)

In [129]:
# for now, we just need 4 columns

wrds_bridge = wrds_bridge[['CUSIP', 'GVKEY', 'LINKDT', 'LINKENDDT', 'CONM']]

In [130]:
# see data structure
wrds_bridge.sample(3)


,CUSIP,GVKEY,LINKDT,LINKENDDT,CONM
18792,888303104,28576,08/31/1993,12/22/1997,TITAN HOLDINGS INC
1964,165195108,2983,07/01/1973,08/30/1978,CHESAPEAKE INDUSTRIES INC
25605,65332E101,120593,05/18/1999,11/28/2003,NEXTERA ENTERPRISES INC


In [131]:
# bridge 2
evans_bridge = pd.read_csv('./data/evans_bridge.csv')

In [132]:
evans_bridge.sample(3)

,DealNumber,agvkey,tgvkey
23238,789043040,1706.0,9173.0
95651,2320221020,177224.0,NaN
9544,624668040,13197.0,NaN


In [33]:
# AF - BJ

name_lst = [
    'ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC', 'AUPBL', 'AUPNAMES', 'AUPPUB',
    'BLOCK','CREEP','DA','DE','STATC','SYNOP','VAL','PCTACQ','PSOUGHTOWN','PSOUGHT','PHDA','PCTOWN','PSOUGHTT','PRIVATIZATION','DEAL_NO',
    'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC', 'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB'    
]

before concat all data, manually convert all date on excel file to "YY-mm-dd" format.

Guide: select all date var --> right-click --> Cell format --> Date --> 2012-03-14 --> OK

![](https://cdn.mathpix.com/snip/images/58hcVJ3qFlC446Ns4SaJKtm-UroEqUKyqu4oCnTWhKY.original.fullsize.png)

In [19]:
def concat_data(st, end):
    df_l = []
    for year in range(st, end+1, 1):
        df = pd.read_excel(f"./data/SDC/{year}.xlsx", header=1, engine='openpyxl')
        #df = df.drop(df.columns[4], axis=1) # this column is duplicate with column 3 
        #print(len(df.columns))
        df.columns = name_lst
        
        # check date var loading ok
        check = df[df['DA'] == datetime.time(0, 0)]
        if check.shape[0] == 0 :
            print('date variables loading ok \n')
        else:
            print('date variables loading fail, please manually check. number of failed records: ', check.shape[0])
        
        df_l.append(df)
        print(f'{year} data shape:', df.shape)
        del df
    df = pd.concat(df_l)
    return df

In [20]:
sdc_df = concat_data(s_year, e_year)

date variables loading ok 

1997 data shape: (13255, 35)
date variables loading ok 

1998 data shape: (15081, 35)
date variables loading ok 

1999 data shape: (13203, 35)
date variables loading ok 

2000 data shape: (12610, 35)
date variables loading ok 

2001 data shape: (8771, 35)
date variables loading ok 

2002 data shape: (7943, 35)
date variables loading ok 

2003 data shape: (8573, 35)
date variables loading ok 

2004 data shape: (9704, 35)
date variables loading ok 

2005 data shape: (10524, 35)
date variables loading ok 

2006 data shape: (11802, 35)
date variables loading ok 

2007 data shape: (12866, 35)
date variables loading ok 

2008 data shape: (11174, 35)
date variables loading ok 

2009 data shape: (8965, 35)
date variables loading ok 

2010 data shape: (9365, 35)
date variables loading ok 

2011 data shape: (9859, 35)
date variables loading ok 

2012 data shape: (9772, 35)
date variables loading ok 

2013 data shape: (10060, 35)
date variables loading ok 

2014 data s

In [21]:
sdc_df = sdc_df.reset_index(drop=True) 

In [22]:
print(f'saving sdc table ranging from {s_year} to {e_year} ...')
sdc_df.to_pickle(pjoin(tmp_data_path , f'sdc_{s_year}_{e_year}.pickle'))

saving sdc table ranging from 1997 to 2020 ...


In [134]:
# You can run code start from here if merging sdf_df is unnecessary

if os.path.isfile(pjoin(tmp_data_path , f'sdc_{s_year}_{e_year}.pickle')):
    sdc_df = pd.read_pickle(pjoin(tmp_data_path , f'sdc_{s_year}_{e_year}.pickle'))

## Data Preparing

1. delete obs either `ACU` or `TCU` in full_df is NAs (otherwise we cannot identify the participants of deals)
2. match the format of: time, CUSIP, GVKEY to be the same across three dataset (otherwise may trigger error when merging tables)
    - Time: Timestap
    - CUSIP / GVKEY: string
    - dealnum: string



###  sdc data

1. drop where either `ACU` or `TCU` is Nas
1. fill DEAL_NO NAs to -1
1. change all identifier to `str`; including: `ACU`, `AUP`, `TCU`, `TUP`, `DEAL_NO`, `GVKEY`

In [135]:
sdc_df = sdc_df.dropna(subset=['ACU','TCU']) # actually nothing drops

In [136]:
# deal num
sdc_df['DEAL_NO'] = sdc_df['DEAL_NO'].fillna(-1)
sdc_df['DEAL_NO'] = sdc_df['DEAL_NO'].astype(str)

In [137]:
sdc_df['TCU']  = sdc_df['TCU'].astype('str')
sdc_df['ACU']  = sdc_df['ACU'].astype('str')
sdc_df['TUP']  = sdc_df['TUP'].astype('str')
sdc_df['AUP']  = sdc_df['AUP'].astype('str')

### bridges data

1. match variable type for merging
    1. for CUSIP/GVKEY/DEALNUM, all convert to `string`; 
        - do not keep 0s at front 
            - e.g. `002030` will be curtail to `2030`
    2. for time, all convert to pandas `Timestamp` instance
2. drop na or fill na


#### evans_bridge

for evans_brdge, load as float:
1. fill na as -1
2. convert all var to integer
3. convert all var to string


**so the GVKEY has no 0 at front**

In [138]:
evans_bridge.DealNumber = evans_bridge.DealNumber.fillna(-1)
evans_bridge.tgvkey = evans_bridge.tgvkey.fillna(-1)
evans_bridge.agvkey = evans_bridge.agvkey.fillna(-1)


evans_bridge = evans_bridge.astype('int')
evans_bridge = evans_bridge.astype('str')


#### wrds_bridge

- `GVKEY` and `CUSIP`, load as int; so just convert to str
- no need to worry about NA

In [139]:
wrds_bridge[['GVKEY', 'CUSIP']] = wrds_bridge[['GVKEY', 'CUSIP']].applymap(str)

In [140]:
# covvert 9 digit to 6 digit; 
if len(wrds_bridge.CUSIP[0]) == 9:
    wrds_bridge['CUSIP'] = wrds_bridge.apply(lambda row: row['CUSIP'][:6], axis=1)

since there are lots of "E" in LINKENDDT(still effective presently), LINKENDDT was read as str;

we replace "E" as '12/31/2099' (string type), and then convert to pandas timestamp

In [141]:
wrds_bridge.LINKENDDT[wrds_bridge.LINKENDDT == 'E'] = '12/31/2099'

In [142]:
wrds_bridge['LINKENDDT'] = pd.to_datetime(wrds_bridge['LINKENDDT'],format = '%m/%d/%Y' )

In [143]:
wrds_bridge['LINKDT'] = pd.to_datetime(wrds_bridge['LINKDT'])

#### store bridge to pickle

# Merge

general merging strategy:
1. we take wrds linking as the main linking bridge; evans as secondary.
    1. first use wrds bridge to match GVKEY info to the original SDC data
    2. then use evans_bridge to match GVKEY for those obs which unable to find GVKEY info from wrds_bridge

## check type

In [144]:
#before merge, check all merge variables are in the same type

# gvkey, cusip match

assert type(wrds_bridge.GVKEY[0]) == type(wrds_bridge.CUSIP[0]) == type(evans_bridge.agvkey[0]) 

assert type(evans_bridge.agvkey[0]) == type(evans_bridge.tgvkey[0]) == type(sdc_df.ACU[0]) == type(sdc_df.TCU[0])
        
# deal number match

assert type(sdc_df.DEAL_NO[0]) == type(evans_bridge.DealNumber[0])

# time match

assert type(sdc_df.DA[0]) == type(sdc_df.DE[0]) == type(wrds_bridge.LINKDT[0]) == type(wrds_bridge.LINKENDDT[0])
    
    

## obtain GVKEY from linkings

Rules:
1. use WRDS linking table as primary table, EVANS as secondary
2. drop those no gvkey was returned


reason for rules refering to [Appendix 1](./Appendix1_data_explore.ipynb)

+ save `merged_2` to pickle before checking out Appendix 1!

In [145]:
# 1st merge with wrds bridge
merged_1_1 = wrds_bridge.merge(sdc_df, left_on='CUSIP', right_on = 'ACU', how = 'right')
merged_1_2 = wrds_bridge.merge(merged_1_1, left_on='CUSIP', right_on = 'TCU', how = 'right')

Since math Acquiror before target, here `CUSIP_x` means target's CUSIP; 

`y` == acquirer

In [146]:
# 2nd merge with evans bridge
merged_2 = evans_bridge.merge(merged_1_2, left_on='DealNumber', right_on='DEAL_NO', how = 'right')

In [147]:
merged_2.shape

(300643, 48)

In [148]:
del merged_1_1, merged_1_2

In [149]:
# save merged data for further exploration
print(f'saving original GVKEY merged table ranging from {s_year} to {e_year} ...')
merged_2.to_pickle(pjoin(tmp_data_path , f'merged_ori_{s_year}_{e_year}.pickle'))

saving original GVKEY merged table ranging from 1997 to 2020 ...


In [9]:
# You can run code start from here 

if os.path.isfile(pjoin(tmp_data_path , f'merged_ori_{s_year}_{e_year}.pickle')):
    merged_2 = pd.read_pickle(pjoin(tmp_data_path , f'merged_ori_{s_year}_{e_year}.pickle'))

### filtering:
1. Target and Acquirer must at least successfully match with one table
1. for those records (acquirer or target) that evans bridge did not much successfully, but wrds bridge matched successfully, confirm the `date accounced` of the deal falls in the effective time period of the linking. 

    why use `DA` instead of `DE`? since `DA` has less Nas
    
    
* I chose to use wrds to be my primary linking. If two linking both matched, I will choose the result from wrds.


In [23]:
merged_raw = merged_2.reset_index(drop=True).copy()

define some conditions:

define a new variable: `LINKING_T` and `LINKING_A`

| WRDS has match | WRDS match in valid time | EVANS has match | LINKING |
|----------------|--------------------------|-----------------|---------|
| 1              | 1                        | 1               | 1       |
| 1              | 0                        | 1               | 1       |
| 0              | 0                        | 1               | 1       |
| 1              | 1                        | 0               | 1       |
| 0              | 0                        | 0               | 0       |





In [24]:
# taregt gvkey

cond_t = ( (merged_raw['GVKEY_x'].notnull() ) & (merged_raw['LINKDT_x'] <= merged_raw['DA']) & (merged_raw['LINKENDDT_x'] >= merged_raw['DA']) ) | ((merged_raw['tgvkey'].notnull()) & (merged_raw['tgvkey'] != '-1')) 

merged_raw['LINKING_T'] = np.where(cond_t, '1', '0') # create a new column first 


In [28]:
# acquiror gvkey

cond_a =  ( (merged_raw['GVKEY_y'].notnull()) & (merged_raw['LINKDT_y'] <= merged_raw['DA']) & (merged_raw['LINKENDDT_y'] >= merged_raw['DA']) ) | ((merged_raw['agvkey'].notnull()) & (merged_raw['agvkey'] != '-1')) 

merged_raw['LINKING_A'] = np.where(cond_a, '1', '0') # create a new column first 


In [36]:
# wrds as primary, evans as secondary

def get_tgvkey(row):
    # for target
    if pd.notna(row['GVKEY_x']):
        return row['GVKEY_x']        
    elif pd.notna(row['tgvkey']):
        return row['tgvkey']
    else: 
        print('something is wrong')
        
 

def get_agvkey(row):
    # for target
    if pd.notna(row['GVKEY_y']):
        return row['GVKEY_y']        
    elif pd.notna(row['agvkey']):
        return row['agvkey']
    else:
        print('something is wrong')



In [38]:
merged_bothok = merged_raw[(merged_raw['LINKING_A'] == '1') & (merged_raw['LINKING_T'] == '1')]

merged_bothok['TGVKEY'] = merged_bothok.apply(get_tgvkey, axis=1)
merged_bothok['AGVKEY'] = merged_bothok.apply(get_agvkey, axis=1)


<ipython-input-38-41749d380e89>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_bothok['TGVKEY'] = merged_bothok.apply(get_tgvkey, axis=1)
<ipython-input-38-41749d380e89>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_bothok['AGVKEY'] = merged_bothok.apply(get_agvkey, axis=1)


In [39]:
new_name_lst = name_lst + ['AGVKEY', 'TGVKEY']

In [40]:
merged = merged_bothok[new_name_lst]

In [58]:
# merged = merged[ ((merged['AGVKEY'].notnull()) & (merged['TGVKEY'].notnull()))]

**remove those self merge with self**


I am sure we should do that!!!!

In [41]:
merged = merged[merged['AGVKEY'] != merged['TGVKEY']]

In [60]:
del merged_bothok, merged_targetok

In [42]:
print(f'filtered merged data using data from {s_year} to {e_year} = ', merged.shape)

merged.to_pickle(pjoin(tmp_data_path , f'merged_filtered_{s_year}_{e_year}.pickle'))

filtered merged data using data from 1997 to 2020 =  (17902, 37)


In [43]:
merged.columns

Index(['ACU', 'ASIC2', 'ABL', 'ANL', 'APUBC', 'AUP', 'AUPSIC', 'AUPBL',
       'AUPNAMES', 'AUPPUB', 'BLOCK', 'CREEP', 'DA', 'DE', 'STATC', 'SYNOP',
       'VAL', 'PCTACQ', 'PSOUGHTOWN', 'PSOUGHT', 'PHDA', 'PCTOWN', 'PSOUGHTT',
       'PRIVATIZATION', 'DEAL_NO', 'TCU', 'TSIC2', 'TBL', 'TNL', 'TPUBC',
       'TUP', 'TUPSIC', 'TUPBL', 'TUPNAMES', 'TUPPUB', 'AGVKEY', 'TGVKEY'],
      dtype='object')

In [52]:
merged.STATC.value_counts()

C     14951
P      1246
W      1119
UN      439
DR       91
I        48
R         5
IW        2
PC        1
Name: STATC, dtype: int64

# Types of Takeover


## Public?
- Although `GVKEY` is mostly for public firm, but some private firm do have `GVKEY` as well.
- However, TNIC would only cover Public firms.

Which means, we could only analysis the following conditions:

| APUBC == Public? | AUPPUB == Public? | TPUBC == Public? | TUPPUB == Public? | mark as                                           |
|------------------|-------------------|------------------|-------------------|---------------------------------------------------|
| 1                | 1                 | 1                | 1                 | 1                                                 |
| 1                | 1                 | 1                | 0                 | 2                                                 |
| 1                | 1                 | 0                | 1                 | 3                                                 |
| 1                | 0                 | 1                | 1                 | 4                                                 |
| 1                | 0                 | 1                | 0                 | 5                                                 |
| 1                | 0                 | 0                | 1                 | 6                                                 |
| 0                | 1                 | 1                | 1                 | 7                                                 |
| 0                | 1                 | 1                | 0                 | 8                                                 |
| 0                | 1                 | 0                | 1                 | 9                                                 |
|                  |                   |                  |                   | all other combination is unanalysiable, mark as 0 |

The analysis the ratio refer to [Appendix 1](./Appendix1_data_explore.ipynb)

**Conclusion**
- `3` and `9` are one type of MA
- `1` and `7` are another type of MA
- drop others






In [53]:
def mark_if_public(row, **kwargs):
    if (row.APUBC == 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB == 'Public'):
        return '1'
    elif (row.APUBC == 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB != 'Public'):
        return '2'
    elif (row.APUBC == 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC != 'Public') & (row.TUPPUB == 'Public'):
        return '3'
    elif (row.APUBC == 'Public') &  (row.AUPPUB != 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB == 'Public'):
        return '4'
    elif (row.APUBC == 'Public') &  (row.AUPPUB != 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB != 'Public'):
        return '5'
    elif (row.APUBC == 'Public') &  (row.AUPPUB != 'Public') & (row.TPUBC != 'Public') & (row.TUPPUB == 'Public'):
        return '6'
    elif (row.APUBC != 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB == 'Public'):
        return '7'
    elif (row.APUBC != 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC == 'Public') & (row.TUPPUB != 'Public'):
        return '8'
    elif (row.APUBC != 'Public') &  (row.AUPPUB == 'Public') & (row.TPUBC != 'Public') & (row.TUPPUB == 'Public'):
        return '9'
    else:
        return '0'

In [54]:
merged['PUB_COND'] = merged.apply(mark_if_public, axis=1)

In [56]:
merged_public = merged[merged['PUB_COND'].isin(['1','3','7','9'])].copy()

In [58]:
print(f'pickle merged data with public info from {s_year} to {e_year} = ', merged_public.shape)

merged_public.to_pickle(pjoin(tmp_data_path , f'merged_public_{s_year}_{e_year}.pickle'))

pickle merged data with public info from 1997 to 2020 =  (16649, 38)


# 4 STATC == Completed


Consider `Complete` and `Withdrawn`


In [65]:
merged_modeling = merged_public[merged_public.STATC.isin(['C', 'W'])].copy()

## add Year and SIC

In [69]:
def get_sic(df):
    '''
    df: the sdc table contains sic variable named as `ASIC2`
    
    '''
    x = df.ASIC2.str.split('/')
    x = x.transform(lambda x: x[0] if not isinstance(x, float) else np.nan)
    df['SIC_A'] = x

    x = df.ASIC2.str.split('/')
    x = x.transform(lambda x: x[0] if not isinstance(x, float) else np.nan)
    df['SIC_T'] = x
    
    return df

In [67]:
merged_modeling["YEAR"] = merged_modeling.DA.dt.year

In [70]:
merged_modeling = get_sic(merged_modeling)

In [71]:
print(f'pickle merged data to model from {s_year} to {e_year} = ', merged_modeling.shape)

merged_modeling.to_pickle(pjoin(tmp_data_path , f'merged_model_{s_year}_{e_year}.pickle'))

pickle merged data to model from 1997 to 2020 =  (14932, 41)
